In [1]:
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
from scipy import stats
from scipy.io import loadmat
from sklearn.cluster.bicluster import SpectralBiclustering, SpectralCoclustering

import cPickle
from math import*

In [2]:
z_scores = np.load('raw_z_npdump.dump')
print len(z_scores), len(z_scores[0])

64 3383768


In [3]:
length_of_intervals = 10000
channels = 63
edges = 2016
total_interval = len(z_scores[0])/length_of_intervals + 1
channels_data = [[None for j in range(edges)] for i in range(total_interval)] 
print total_interval, edges

339 2016


In [4]:
def euc_dis(x, y):
    return sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))

def calculate_euclidean(interval):
    index_counter = 0
    j = length_of_intervals * interval
    for start in range(channels):
        for index in range(channels):
            if start <= index:
                if interval != total_interval:
                    channels_data[interval][index_counter] = exp(-euc_dis(z_scores[start][j:j+length_of_intervals],
                                                                     z_scores[index][j:j+length_of_intervals]))
                else:
                    channels_data[interval][index_counter] = exp(-euc_dis(z_scores[start][j:],
                                                                     z_scores[index][j:]))
                index_counter += 1
    print index_counter

In [ ]:
for i in range(total_interval):
    print 'Calculating Interval %s' % i,
    calculate_euclidean(i)

Calculating Interval 0 2016
Calculating Interval 1 2016
Calculating Interval 2 2016
Calculating Interval 3 2016
Calculating Interval 4 2016
Calculating Interval 5

In [14]:
print channels_data[0]

[0.0, 72.97266197792372, 165.43650420874852, 75.9582699696545, 197.98016340471236, 334.8397596272161, 207.79335139919058, 15.21916626841564, 255.00248712834363, 276.1947843952477, 110.80203162985367, 59.7736177191657, 80.44806681962804, 78.68468031122835, 24.826768674891266, 184.08363030596632, 173.8024926135262, 143.97370142487173, 41.27885918950492, 281.7692869940485, 172.3061950532503, 4.700540860045228, 253.27385836923105, 154.74110292329334, 70.01530341909735, 228.78322806579848, 79.2634100129216, 186.52283961094307, 312.2684237937528, 176.7819784880112, 253.9075865467679, 98.17807673891417, 84.68407709855347, 224.16440047448646, 54.341744241285895, 295.1128904896251, 200.6781317700028, 48.316913317253636, 87.94086186134332, 233.337651128959, 264.8450189469343, 10.029475019546146, 190.6959399069445, 82.3005218981594, 160.15124599710273, 19.62957544640381, 164.53271256303938, 183.53739806707796, 208.26209200639082, 209.69508434827543, 6.74305857884055, 204.6487476279198, 88.3171347

In [13]:
pearson_data = np.array(channels_data)
spectral_model = SpectralBiclustering()
spectral_model.fit(pearson_data)
fit_data = pearson_data[np.argsort(spectral_model.row_labels_)]
fit_data = fit_data[:, np.argsort(spectral_model.column_labels_)]
plt.matshow(pearson_data, cmap=plt.cm.Blues)
plt.matshow(fit_data, cmap=plt.cm.Blues)
plt.show()

ValueError: array must not contain infs or NaNs

In [ ]:
import json as simplejson
with open('media/euc_z_score_30s_2016.json', 'w+') as f:
    p = [[float(column) for column in row] for row in pearson_data]
    f.write(simplejson.dumps({'name': 's5d2nap', 'data': p}))